In [1]:
# import dependencies
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')
# %matplotlib notebook  <----- use for better visuals, but may impede view
#pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [2]:
#Read CSV
Youtube_Data = pd.read_csv('US_youtube_trending_data.csv')

# Convert CSV to Data Frame
Youtube_Data = pd.DataFrame(Youtube_Data)

# preview Data Frame
Youtube_Data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'US_youtube_trending_data.csv'

In [ ]:
# Preview last 5 rows of Data Frame
Youtube_Data.tail()

In [ ]:
Youtube_Data.columns

In [ ]:
# Observe Data Types
Youtube_Data.dtypes

In [ ]:
# Number of Records in data
len(Youtube_Data)

In [ ]:
print ("Rows     : " , Youtube_Data.shape[0]) #Displays numbers of rows .
print ("Columns  : " , Youtube_Data.shape[1]) # and column our dataset contains.
print ("\nFeatures : \n", Youtube_Data.columns.tolist())#displays column names
print ("\nMissing values :  ", Youtube_Data.isnull().sum().values.sum())   #find missing values
print ("\nUnique values :  \n", Youtube_Data.nunique())  # Count distinct observations

In [ ]:
# finding number of null records for each column
Youtube_Data.isna().sum()

In [ ]:
# Dropping unecessary columns
Youtube_Data = Youtube_Data.drop(columns=['tags','comment_count','thumbnail_link','comments_disabled','ratings_disabled','description'])

In [ ]:
# Renaming columns
Youtube_Data = Youtube_Data.rename(columns={'likes':'Likes','dislikes':'Dislikes',
                                            'channelTitle':'Channel Title','channelTitle':'Channel Title',
                                           'video_id':'Video ID','channelId':'Channel ID'})

In [ ]:
# First five rows of updated Data Frame
Youtube_Data.head()

In [ ]:
# add new column that would contain latest view_count for the particular video.
#since number of views keep on increasing everyday , we will be using only single observation against a video id.
Youtube_Data['count_max_view'] = Youtube_Data.groupby(['Video ID'])['view_count'].transform(max)

In [ ]:
# Renaming Data Frame
Youtube_Unique_Video_Id = Youtube_Data

In [ ]:
#changing published date , trending_date to datetime datatype.
Youtube_Unique_Video_Id.publishedAt= pd.to_datetime(Youtube_Unique_Video_Id.publishedAt)
Youtube_Unique_Video_Id.trending_date= pd.to_datetime(Youtube_Unique_Video_Id.trending_date)

In [ ]:
#Adding column for year and month to the dataframe
Youtube_Unique_Video_Id['Year']=Youtube_Unique_Video_Id['publishedAt'].dt.year
Youtube_Unique_Video_Id['Month']=Youtube_Unique_Video_Id.publishedAt.dt.month

In [ ]:
# make a new dataframe with only single observations for each id.
Youtube_Unique_Video_Id = Youtube_Data.loc[Youtube_Data.view_count == Youtube_Data.count_max_view]
# Youtube_Unique_Video_Id.loc[Youtube_Unique_Video_Id.Year == 2021, 'title'].count()

In [ ]:
# Use Youtube_Unique_Video_Id for main dataset

In [ ]:
# Number of unique records in each column
Youtube_Unique_Video_Id.nunique()

In [ ]:
Youtube_Unique_Video_Id.duplicated(subset=['Video ID'])

In [ ]:
Reocurring_Trending_Videos = Youtube_Unique_Video_Id.groupby(['Video ID']).count().sort_values(by='title',ascending=False)
Reocurring_Trending_Videos.head(10)

In [ ]:
Youtube_Unique_Video_Id.loc[Youtube_Unique_Video_Id['Video ID'] == '69KrkMpvZdg']

In [ ]:
# display data frame first five rows
Youtube_Unique_Video_Id.head()

In [ ]:
# Basic Statistics for Data Frame
Youtube_Unique_Video_Id.describe().applymap('{:,.2f}'.format)

In [ ]:
# gives us various count of video in each channel category
Youtube_Unique_Video_Id.groupby("categoryId").size().reset_index(name="View Count") \
    .sort_values("View Count", ascending=False).head(20)

In [ ]:
#Getting category for the category file
#we need id and title from this file which are under items tag and snippet contain title
with open("US_category_id.json") as f:
    categories = json.load(f)["items"]
cat_dict = {}
category=[]
for cat in categories:
    category.append([cat["id"],cat["snippet"]["title"]])

In [ ]:
df_category = pd.DataFrame(category,columns=['categoryId','category']) 
df_category.head(20)

In [ ]:
df_category.categoryId=df_category.categoryId.astype('int64')
#merge YT Data Frame and df_category to get category names
df_merge = Youtube_Unique_Video_Id.merge(df_category, on = 'categoryId', how = 'inner')
df_merge.head()

In [ ]:
# Create summary chart for video counts by categories
agg_func_count = {'view_count': ['count', 'nunique', 'size']}
df_merge.groupby(['category']).agg(agg_func_count)

#change to clean data frame name
Youtube_Data_Updated_Merged = df_merge

In [ ]:
# Youtube Data for 2020
YT_Data_2020 = df_merge.loc[(df_merge['Year']) >= 2020]

In [ ]:
# Counts of video in each category published in 2020
YT_2020_Category_Summary = {'view_count': ['count', 'nunique', 'size']}
YT_2020_Category_Summary = YT_Data_2020.groupby(['category']).agg(agg_func_count).sort_values(by=('view_count','count'),ascending=False)

In [ ]:
# mean, median, mode, and average of the video categories count list (max, min, mode)
# Calculate total likes
# Avaerage of likes against total volume of videos
# Correlation between like count and volume of views
YT_2020_Category_Summary

In [ ]:
df_merge.min()

In [ ]:
# creating same summary table, but instead using .loc to grab data only from time period needed (08/20-08/21)

# Defining COVID Date Range for Data
COVID_Start_Date = '2020-08-01'
COVID_End_Date = '2021-08-31'

# Filtering Youtube Data for 08/2020-08/2021 (Project's COVID period)
YT_Data_COVID = df_merge.loc[(df_merge['publishedAt'] >= COVID_Start_Date) & (df_merge['publishedAt'] <= COVID_End_Date)]
                            

# Counts of video in each category published in 2020
YT_COVID_Category_Summary = {'view_count': ['count']}
YT_COVID_Category_Summary = YT_Data_COVID.groupby(['category']).agg(agg_func_count).sort_values(by=('view_count','count'),ascending=False)

# mean, median, mode, and average of the video categories count list (max, min, mode)
# Calculate total likes
# Avaerage of likes against total volume of videos
# Correlation between like count and volume of views

# Convert to Data Frame
YT_COVID_Category_Summary = pd.DataFrame(YT_COVID_Category_Summary)

YT_COVID_Category_Summary

In [ ]:
# creating same summary table, but instead using .loc to grab data only from Post-COVID

# Filtering Youtube Data for 09/2021-07/2022 (Project's post-COVID period)
YT_Data_PostCOVID = df_merge.loc[(df_merge['publishedAt'] > COVID_End_Date)]

In [ ]:
# Top 10 videos by category for all data (08/2020-07/25/22)
df_merge.groupby(df_merge['category']).size().sort_values(ascending=True).head(10).plot(kind='bar',color='red')
plt.title('Top 10 Video Categories By Video Uploads 08/2020-7/2022')
plt.xlabel('Category')
plt.ylabel('Video Count')
plt.xticks(rotation=45,ha='right',rotation_mode='anchor')
plt.show()
plt.best

In [ ]:
# Top 10 videos by category for COVID time-frame data (08/2020-08/2021)
YT_Data_COVID.groupby(YT_Data_COVID['category']).size().sort_values(ascending=True).head(10).plot(kind='bar',color='red')
plt.title('Top 10 Video Categories By Video Uploads 08/2020-08/2021')
plt.xlabel('Category')
plt.ylabel('Video Count')
plt.xticks(rotation=45,ha='right',rotation_mode='anchor')
plt.show()

In [ ]:
# Top 10 videos by category for post-COVID time-frame data (09/2021-07/2022)
YT_Data_PostCOVID.groupby(YT_Data_PostCOVID['category']).size().sort_values(ascending=True).head(10).plot(kind='bar',color='red')
plt.title('Top 10 Video Categories By Video Uploads 09/2021-07/2022')
plt.xlabel('Category')
plt.ylabel('Video Count')
plt.xticks(rotation=45,ha='right',rotation_mode='anchor')
plt.show()

In [ ]:
# Showing the average view count of popular trending videos by category during COVID scope (08/2020-08/2021)
COVID_Category_Mean_View_Count = \
YT_Data_COVID.groupby('category').mean('view_count')['view_count'].sort_values(ascending=True).map('{:,.0f}'.format)

COVID_Category_Mean_View_Count

In [ ]:
# Showing the average view count of popular trending videos by category during COVID scope (08/2020-08/2021)
plt.bar(COVID_Category_Mean_View_Count.index, COVID_Category_Mean_View_Count.values,color='red')
# Setting axis labels
plt.title('Average Viral Video View Count by Category 08/2020-08/2021')
plt.xticks(rotation='vertical')
# plt.xlabel('Category')
plt.ylabel('Average View Count')
plt.show()

In [ ]:
# Showing the average view count of popular trending videos by category after COVID scope (09/2021-07/2022)
PostCOVID_Category_Mean_View_Count = \
YT_Data_PostCOVID.groupby('category').mean('view_count')['view_count'].sort_values(ascending=True).map('{:,.0f}'.format)

PostCOVID_Category_Mean_View_Count

In [ ]:
# Showing the average view count of popular trending videos by category after COVID scope (09/2021-07/2022)
plt.bar(PostCOVID_Category_Mean_View_Count.index, PostCOVID_Category_Mean_View_Count.values,color='red')
# Setting axis labels
plt.title('Average Viral Video View Count by Category 09/2021-07/2022')
plt.xticks(rotation='vertical')
# plt.xlabel('Category')
plt.ylabel('Average View Count')
plt.show()

In [ ]:
# Video Categories Averages Data Frame for full data set (08/2020-07/2022)
Category_viewCount_Likes_Dislikes_All = \
df_merge.groupby('category').mean().sort_values(by=['Likes','Dislikes'],ascending=False)#.map('{:,.0f}'.format)

Category_viewCount_Likes_Dislikes_All = pd.DataFrame(Category_viewCount_Likes_Dislikes)

Category_viewCount_Likes_Dislikes_All = Category_viewCount_Likes_Dislikes.round(0)

In [ ]:
# Create Scatter plot and show correlation for full data set (08/2020-07/2022)
x_values = Category_viewCount_Likes_Dislikes['view_count']
y_values = Category_viewCount_Likes_Dislikes['Likes']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(3,3),fontsize=15,color="red")
plt.title('Average Like Count by Average Total Category View Count 08/2020-07/2022')
plt.xlabel('Category Average View Count')
plt.ylabel('Average Likes Count')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# Video Categories Averages Data Frame for COVID scope data (08/2020-08/2021)
Category_viewCount_Likes_Dislikes_COVID = \
YT_Data_COVID.groupby('category').mean().sort_values(by=['Likes','Dislikes'],ascending=False)#.map('{:,.0f}'.format)

Category_viewCount_Likes_Dislikes_COVID = pd.DataFrame(Category_viewCount_Likes_Dislikes_COVID)

Category_viewCount_Likes_Dislikes_COVID = Category_viewCount_Likes_Dislikes_COVID.round(0)

In [ ]:
# Create Scatter plot and show correlation for COVID data (08/2020-08/2021)
x_values = Category_viewCount_Likes_Dislikes_COVID['view_count']
y_values = Category_viewCount_Likes_Dislikes_COVID['Likes']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(3,3),fontsize=15,color="red")
plt.title('Average Like Count by Average Total Category View Count 08/2020-08/2021')
plt.xlabel('Category Average View Count')
plt.ylabel('Average Likes Count')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
# Video Categories Averages Data Frame for post-COVID scope data (09/2021-07/2022)
Category_viewCount_Likes_Dislikes_PostCOVID = \
YT_Data_PostCOVID.groupby('category').mean().sort_values(by=['Likes','Dislikes'],ascending=False)#.map('{:,.0f}'.format)

Category_viewCount_Likes_Dislikes_PostCOVID = pd.DataFrame(Category_viewCount_Likes_Dislikes_PostCOVID)

Category_viewCount_Likes_Dislikes_PostCOVID = Category_viewCount_Likes_Dislikes_PostCOVID.round(0)

In [ ]:
# Create Scatter plot and show correlation for COVID data (09/2021-07/2022)
x_values = Category_viewCount_Likes_Dislikes_PostCOVID['view_count']
y_values = Category_viewCount_Likes_Dislikes_PostCOVID['Likes']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(3,3),fontsize=15,color="red")
plt.title('Average Like Count by Average Total Category View Count 09/2021-07/2022')
plt.xlabel('Category Average View Count')
plt.ylabel('Average Likes Count')
print(f"The r-squared is: {rvalue**2}")
plt.show()